### Part C. 고급 응용 분석 - 토픽 모델링

- Python3.12.6 사용중

- 필요한 라이브러리 설치.

In [2]:
!pip install -q pyLDAvis tomotopy --upgrade


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install gdown

  Using cached gdown-5.2.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached charset_normalizer-3.4.3-cp312-cp312-macosx_10_13_universal2.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
Using cached gdown-5.2.0-py3-none-any.whl (18 kB)
Using cached filelock-3.19.1-py3-none-any.whl (15 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached certifi-2025.8.3-py3-none-any.whl (161 kB)
Using cached charset_normalizer-3.4.3-cp312-cp312-macosx_10_13_universal2.whl (205 kB)
Using cached idna-3.10-

In [5]:
import gdown  # 구글 드라이브 파일 다운로드에 활용
import pandas as pd  # 데이터프레임 처리, CSV/JSON 입출력 등
from typing import List, Dict, Any, Tuple, Optional  # 타입 힌트 제공
#pyLDAvis과 numpy 라이브러리 충돌문제 해결, 런타임 재실행없이 실행하기 위한 디버깅코드
import warnings
import numpy as np
import pyLDAvis
import tomotopy as tp
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [6]:
# 1) Google Drive 파일 ID
file_id = "1fgVYI6UrVs96zO6sAFcirwJhPCLvZQRv" #임이로 기존 데이터 사용.
output = "IRI이미지형용사_definitions_tokens_kiwi.csv"

# 2) 파일 다운로드 (gdown 사용)
gdown.download(f"https://drive.google.com/uc?id={file_id}", output, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1fgVYI6UrVs96zO6sAFcirwJhPCLvZQRv
To: /Users/irolim/Documents/한중연/4차시/논자시/exam_2025/IRI이미지형용사_definitions_tokens_kiwi.csv
100%|██████████| 828k/828k [00:00<00:00, 1.38MB/s]


'IRI이미지형용사_definitions_tokens_kiwi.csv'

- 말뭉치 준비

In [7]:
def build_corpus_for_topic(token_df: pd.DataFrame, token_col: str = "norm_token", min_len: int = 2) -> Tuple[List[List[str]], List[int]]:
    """
    문서별 토큰 리스트를 반환
    - min_len: 최소 토큰 길이 (짧은 조사/불용어 제거용)
    """
    # 만약 token_df에 "doc_id" 컬럼이 없으면
    # definition 단위로 그룹을 묶어 새로운 doc_id 생성
    if "doc_id" not in token_df.columns:
        token_df["doc_id"] = token_df.groupby("definition").ngroup()

    docs: List[List[str]] = [] # 문서별 토큰 리스트
    doc_ids: List[int] = [] # 문서 ID 리스트
    # doc_id별 그룹화 후 토큰 모으기
    for doc_id, group in token_df.groupby("doc_id"):
        # NaN 제거 → 문자열 변환 → 리스트화
        # min_len보다 짧은 토큰은 제거 (예: "은", "가" 같은 조사)
        tokens = [t for t in group[token_col].dropna().astype(str).tolist() if len(t) >= min_len]
        # 유효 토큰이 존재하는 문서만 결과에 추가
        if tokens:
            docs.append(tokens) # 문서 단위 토큰 리스트
            doc_ids.append(doc_id) # 해당 문서 ID 저장
    # (문서별 토큰 리스트, 문서 ID 리스트) 반환
    return docs, doc_ids

- LDA 학습

In [8]:
def train_lda(docs: List[List[str]], num_topics: int = 10, seed: int = 42, min_cf: int = 3, rm_top: int = 0, iterations: int = 200) -> tp.LDAModel:
    """
    tomotopy LDA 모델 학습
    - num_topics: 토픽 개수
    - min_cf: 최소 출현 빈도 (저빈도 단어 제거)
    - rm_top: 상위 몇 개의 고빈도 단어 제거
    - iterations: 학습 반복 횟수
    """
    # LDA 모델 초기화
    mdl = tp.LDAModel(
        k=num_topics,  # 토픽 개수
        seed=seed,     # 랜덤 시드
        min_cf=min_cf, # 최소 출현 빈도
        rm_top=rm_top  # 상위 고빈도 단어 제거
    )
    # 문서별 토큰 리스트를 모델에 추가
    for doc in docs:
        mdl.add_doc(doc)
    # 모델 기본 정보 출력
    print(f"[LDA] 총 문서 수: {len(mdl.docs)}, 어휘 수: {mdl.num_vocabs}, 토픽 수: {mdl.k}")

    # 지정된 반복(iterations) 동안 학습 진행
    # 20회 단위로 끊어서 perplexity(혼잡도) 출력
    for i in range(0, iterations, 20):
        mdl.train(20)
        print(f"Iteration: {i+20}\tPerplexity: {mdl.perplexity:.4f}")


    return mdl

- 최적 토픽 수 결정 (Perplexity / Coherence)

In [9]:
def find_optimal_k(docs: List[List[str]], k_range: range, iterations: int = 100) -> pd.DataFrame:
    """
    LDA에서 최적 토픽 개수(k)를 찾기 위한 함수

    매개변수:
    - docs: 문서별 토큰 리스트 (list of list 형태)
    - k_range: 실험할 토픽 개수 범위 (예: range(5, 16, 1))
    - iterations: 각 k값마다 학습 반복 횟수 (기본 100)

    반환값:
    - 결과 DataFrame (컬럼: k, perplexity, coherence)
      - perplexity: 혼잡도 (낮을수록 모델이 데이터를 잘 설명)
      - coherence: 토픽 일관성 점수 (높을수록 해석 가능성이 높음)
    """
    results = []

    # 주어진 범위의 k 값에 대해 반복
    for k in k_range:
        # LDA 모델 초기화 (k개 토픽, 최소 출현 빈도=3)
        mdl = tp.LDAModel(k=k, seed=42, min_cf=3)

        # 문서 추가
        for doc in docs:
            mdl.add_doc(doc)

        # 모델 학습
        mdl.train(iterations)

        # Perplexity 계산 (모델의 적합도, 낮을수록 좋음)
        perp = mdl.perplexity

        # Coherence 계산 (토픽 해석 가능성, 높을수록 좋음)
        coh = tp.coherence.Coherence(mdl, coherence='c_v').get_score()

        # 결과 저장
        results.append({"k": k, "perplexity": perp, "coherence": coh})

        # 중간 결과 출력
        print(f"k={k} -> Perplexity={perp:.4f}, Coherence={coh:.4f}")

    # k별 결과를 DataFrame으로 반환
    return pd.DataFrame(results)


- 토픽 주요 단어 추출

In [10]:
# 전체 토픽(k=mdl.k)에 대해 순회하며 get_topic_words 호출
def get_topic_words(mdl: tp.LDAModel, top_n: int = 10) -> Dict[int, List[Tuple[str, float]]]:
    return {k: mdl.get_topic_words(k, top_n=top_n) for k in range(mdl.k)}


- 문서별 토픽 분포

In [11]:
# 문서-토픽 분포(Document-Topic Distribution) 추출 함수
def get_doc_topic_dist(mdl: tp.LDAModel) -> pd.DataFrame:
  # 각 문서(doc)에 대해 토픽 분포 벡터 추출
    doc_topics = [doc.get_topic_dist() for doc in mdl.docs]
    # DataFrame으로 변환, 컬럼명은 topic_0, topic_1, ...
    return pd.DataFrame(doc_topics, columns=[f"topic_{i}" for i in range(mdl.k)])

- pyLDAvis 시각화 (tomotopy.visualize 없을 때도 동작)

In [12]:
def _prepare_ldavis_fallback(mdl: tp.LDAModel):
    """
    tomotopy.visualize 모듈이 없는 경우(구버전 등)를 대비한 pyLDAvis 준비 함수
    - wid가 int 또는 str인 경우 모두 처리
    - term_frequency, doc_lengths, topic_term_dists, doc_topic_dists 구성
    """
    # --- 어휘 목록 및 매핑 ---
    vocab = list(mdl.used_vocabs)        # 모델이 실제 사용한 단어 목록
    V = len(vocab)                       # 전체 어휘 수
    vocab2id = {w: i for i, w in enumerate(vocab)}  # 단어→인덱스 매핑

    # --- term_frequency (길이 V): 전체 코퍼스에서 각 단어의 출현 빈도 ---
    term_frequency = np.zeros(V, dtype=np.int64)
    for d in mdl.docs:
        try:
            pairs = d.get_words()  # (wid, cnt) 형태 리스트
        except AttributeError:
            pairs = None

        if pairs is not None:
            for wid, cnt in pairs:
                # wid가 int/np.integer일 때
                if isinstance(wid, (int, np.integer)):
                    if 0 <= wid < V:
                        term_frequency[wid] += int(cnt)
                # wid가 str(토큰)일 때
                elif isinstance(wid, str):
                    idx = vocab2id.get(wid)
                    if idx is not None:
                        term_frequency[idx] += int(cnt)
        else:
            # 구형 tomotopy에서는 d.words 속성이 wid 리스트일 수 있음
            for wid in getattr(d, "words", []):
                if isinstance(wid, (int, np.integer)):
                    if 0 <= wid < V:
                        term_frequency[wid] += 1
                elif isinstance(wid, str):
                    idx = vocab2id.get(wid)
                    if idx is not None:
                        term_frequency[idx] += 1

    # --- topic_term_dists (K x V): 각 토픽의 단어 분포 φ ---
    topic_term = np.vstack([
        np.asarray(mdl.get_topic_word_dist(k), dtype=np.float64)
        for k in range(mdl.k)
    ])
    topic_term = (topic_term + 1e-12) / (topic_term + 1e-12).sum(axis=1, keepdims=True)

    # --- doc_topic_dists (D x K): 각 문서의 토픽 분포 θ ---
    doc_topic = np.vstack([
        np.asarray(doc.get_topic_dist(), dtype=np.float64)
        for doc in mdl.docs
    ])
    doc_topic = (doc_topic + 1e-12) / (doc_topic + 1e-12).sum(axis=1, keepdims=True)

    # --- 문서 길이 계산 (단어 수) ---
    doc_lengths = []
    for d in mdl.docs:
        try:
            pairs = d.get_words()
            doc_lengths.append(int(sum(int(cnt) for _, cnt in pairs)))
        except AttributeError:
            wl = getattr(d, "words", [])
            doc_lengths.append(int(len(wl)))

    # pyLDAvis용 데이터 준비
    return pyLDAvis.prepare(
        topic_term_dists=topic_term,
        doc_topic_dists=doc_topic,
        doc_lengths=doc_lengths,
        vocab=vocab,
        term_frequency=term_frequency,
        sort_topics=False
    )


def visualize_lda(mdl: tp.LDAModel, outfile: str = "토픽모델_시각화.html"):
    """
    LDA 토픽 모델 시각화 함수
    1순위: tomotopy.visualize 모듈이 있으면 사용
    2순위: _prepare_ldavis_fallback()으로 pyLDAvis 시각화 준비
    - 결과는 HTML 파일로 저장
    - 기본 저장 파일명: '토픽모델_시각화.html' (한글 파일명)
    """
    try:
        import tomotopy.visualize as tpvis
        prepared = tpvis.prepare(mdl)
    except ImportError:
        prepared = _prepare_ldavis_fallback(mdl)

    # pyLDAvis 시각화 결과를 HTML 파일로 저장
    pyLDAvis.save_html(prepared, outfile)
    print(f"✅ 토픽 모델 시각화 저장 완료: {outfile}")
    return prepared


- 함수 실행 (문서별 토픽 분포, 시각화.html)

In [13]:
if __name__ == "__main__":
    # Part B에서 저장한 token_df 불러오기
    token_df = pd.read_csv(output, encoding="utf-8-sig")

    # 'norm_token' 컬럼이 없다면 lemma/form 기반으로 생성
    if "norm_token" not in token_df.columns:
        token_df["norm_token"] = token_df["tok_lemma"].fillna(token_df["tok_form"])

    # 1) 말뭉치 생성 (문서별 토큰 리스트와 ID)
    docs, doc_ids = build_corpus_for_topic(token_df, token_col="norm_token", min_len=2)

    # 2) 최적 토픽 수 탐색 (예: 5~15 범위)
    results_df = find_optimal_k(docs, k_range=range(5, 16), iterations=100)
    print("\n[최적 토픽 수 후보]")
    print(results_df)

    # 3) LDA 모델 학습 (예: k=8 선택)
    mdl = train_lda(docs, num_topics=8, iterations=200)

    # 4) 각 토픽 주요 단어 출력
    topic_words = get_topic_words(mdl, top_n=10)
    for k, words in topic_words.items():
        print(f"\n[토픽 {k}]")
        print(", ".join([f"{w}:{round(p,3)}" for w, p in words]))

    # 5) 문서별 토픽 분포 저장 (CSV, 한글 파일명)
    doc_topic_df = get_doc_topic_dist(mdl)
    doc_topic_df.to_csv("문서별_토픽분포.csv", index=False, encoding="utf-8-sig")
    print("\n문서별 토픽 분포 저장 완료: 문서별_토픽분포.csv")

    # 6) 시각화 결과 저장 (HTML, 한글 파일명)
    vis = visualize_lda(mdl, outfile="토픽모델_시각화.html")
    print("\n토픽 모델 시각화 저장 완료: 토픽모델_시각화.html")


/var/folders/cd/mj8pxhhd6ml7kpdhggf19l_80000gn/T/ipykernel_46708/3608272736.py:27: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  mdl.train(iterations)


k=5 -> Perplexity=287.2808, Coherence=0.6656
k=6 -> Perplexity=290.8404, Coherence=0.7309
k=7 -> Perplexity=279.0569, Coherence=0.7686
k=8 -> Perplexity=292.7083, Coherence=0.7855
k=9 -> Perplexity=284.5378, Coherence=0.8101
k=10 -> Perplexity=288.5440, Coherence=0.8074
k=11 -> Perplexity=290.1807, Coherence=0.8186
k=12 -> Perplexity=292.3652, Coherence=0.8316
k=13 -> Perplexity=287.2897, Coherence=0.8376
k=14 -> Perplexity=306.6374, Coherence=0.8547
k=15 -> Perplexity=278.8317, Coherence=0.8611

[최적 토픽 수 후보]
     k  perplexity  coherence
0    5  287.280758   0.665609
1    6  290.840389   0.730853
2    7  279.056931   0.768597
3    8  292.708309   0.785465
4    9  284.537838   0.810142
5   10  288.543976   0.807415
6   11  290.180735   0.818647
7   12  292.365160   0.831590
8   13  287.289675   0.837615
9   14  306.637365   0.854724
10  15  278.831727   0.861144
[LDA] 총 문서 수: 491, 어휘 수: 0, 토픽 수: 8
Iteration: 20	Perplexity: 353.1553
Iteration: 40	Perplexity: 322.2671
Iteration: 60	Perpl

/var/folders/cd/mj8pxhhd6ml7kpdhggf19l_80000gn/T/ipykernel_46708/2143239419.py:25: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  mdl.train(20)


✅ 토픽 모델 시각화 저장 완료: 토픽모델_시각화.html

토픽 모델 시각화 저장 완료: 토픽모델_시각화.html
